# Experiments

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

MODEL_NAME = "gpt-3.5-turbo"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


### Experiment

In [3]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "Dataset for RAG App"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

e:\MAT496-LangSmith\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'gpt-4o-a229d970' at:
https://smith.langchain.com/o/55075c61-fb01-4762-bfb5-95638bc62c05/datasets/d5760ca9-0eaa-493c-909f-382fe0e64af3/compare?selectedSessions=364bdb6e-be80-46d7-bb74-e1d375ae7fd2




14it [00:50,  3.59s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I make a map?,I don't know.,None,"To create a map in Minecraft, first craft a co...",1,2.496886,3d32f633-70f6-478b-b768-a1e163e021e7,b7c12865-3b77-4807-b78d-80b83a38b49f
1,What are Nether portals used for?,I don't have that information.,None,Nether portals are used to access the Nether d...,1,2.828693,0220929e-9638-485b-bbaa-f5a5bfaf2b63,dc40fd84-4c2b-462d-bb56-f06f6dab508a
2,How do I enchant items in Minecraft?,"In Minecraft, you can enchant items using an e...",None,"To enchant items in Minecraft, you need to cra...",1,3.033264,b5d20f18-7ac5-4d5f-a9a4-2e079cf688a5,f35e0c2c-b499-4db4-b784-863bfc43f331
3,How do I gather resources in Minecraft?,"To gather resources in Minecraft, start by pun...",None,"To gather resources in Minecraft, you should s...",1,3.059136,d50ee8a9-f678-40ea-b4bc-5a35783fdfac,ad03bbd0-4890-4f38-9d70-793a9f5e2d52
4,How do I fight mobs?,I don't know.,None,"Use swords or bows. Keep your distance, attack...",1,1.189720,2eb6643d-1011-4ab9-b92a-24f5c7a61441,f4960236-a1e7-4278-b541-fbb8f3efd88a
5,What’s the difference between Survival and Cre...,Survival mode in games typically involves mana...,None,"Survival mode requires gathering resources, ma...",0,3.263787,34aaed32-1485-4133-a1a4-73021114d881,e395e2b5-6f1f-422a-9208-3a722d50adde
6,How do I mine for diamonds?,I don't know.,None,"Dig down to levels 11–12 underground, mine car...",1,1.319250,3f3ca99b-3f17-4bd8-bdfd-bce4cbc03086,bb830e64-7bd0-434b-b4c9-cb2a93c3e92d
7,How do I start playing Minecraft?,"To start playing Minecraft, first, you need to...",None,"First, create a new world in Survival or Creat...",0,3.398732,98bfec98-72ed-4362-b84c-45d6910fe51e,113272a0-f220-48df-931a-e2d6c900a8d4
8,How do I craft items?,I don't have information on crafting items. It...,None,"Open your inventory (E), place materials in th...",0,2.952605,a81fb131-6a89-44fe-9596-5ae2e39f525f,2dece1a3-edff-4e69-b0f1-837642189364
9,How do I explore safely?,I don't have specific information on exploring...,None,"Carry food, torches, weapons, and armor. Avoid...",1,1.672124,ad4965fd-670b-4d7b-ade3-e126ac213d9c,75db0dd7-5952-4b5b-91a1-97b671849ef8


### Modifying your Application

In [6]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-5029d3a3' at:
https://smith.langchain.com/o/55075c61-fb01-4762-bfb5-95638bc62c05/datasets/d5760ca9-0eaa-493c-909f-382fe0e64af3/compare?selectedSessions=eeb48d15-2278-494b-9a8d-ff21fe580b7b




14it [00:39,  2.84s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I make a map?,I don't know the answer to that question.,None,"To create a map in Minecraft, first craft a co...",1,1.248552,3d32f633-70f6-478b-b768-a1e163e021e7,6fd1d1f5-1ca1-473f-b18d-510fd2b7e803
1,What are Nether portals used for?,I don't know the answer to that question based...,None,Nether portals are used to access the Nether d...,1,1.226017,0220929e-9638-485b-bbaa-f5a5bfaf2b63,21680fcd-32e4-4edc-a6f8-880e4081094c
2,How do I enchant items in Minecraft?,I don't have information on how to enchant ite...,None,"To enchant items in Minecraft, you need to cra...",1,21.943411,b5d20f18-7ac5-4d5f-a9a4-2e079cf688a5,e20ce928-fcc5-4fda-bad4-bbd7828d856e
3,How do I gather resources in Minecraft?,The relevant information from the provided con...,None,"To gather resources in Minecraft, you should s...",1,1.519140,d50ee8a9-f678-40ea-b4bc-5a35783fdfac,3f7d63ed-6a52-41a7-8199-585c6f40499b
4,How do I fight mobs?,I don't have enough information to provide an ...,None,"Use swords or bows. Keep your distance, attack...",1,1.270539,2eb6643d-1011-4ab9-b92a-24f5c7a61441,db94202a-61ea-48ac-8141-a45e8f1a0b35
5,What’s the difference between Survival and Cre...,I don't know.,None,"Survival mode requires gathering resources, ma...",1,1.067785,34aaed32-1485-4133-a1a4-73021114d881,64dfc570-1578-46ae-8959-cf862eed437d
6,How do I mine for diamonds?,I don't know the answer to that question.,None,"Dig down to levels 11–12 underground, mine car...",1,1.147585,3f3ca99b-3f17-4bd8-bdfd-bce4cbc03086,a17e17ba-c7ac-4798-b19f-5d90ff9b27d3
7,How do I start playing Minecraft?,I don't know the answer to that question.,None,"First, create a new world in Survival or Creat...",1,2.609698,98bfec98-72ed-4362-b84c-45d6910fe51e,21d03169-fe99-4e10-ad13-78c533162f58
8,How do I craft items?,I don't have information on how to craft items...,None,"Open your inventory (E), place materials in th...",1,1.068264,a81fb131-6a89-44fe-9596-5ae2e39f525f,650f6a03-9285-4949-a384-ecb8f17b7595
9,How do I explore safely?,I don't have enough information to answer your...,None,"Carry food, torches, weapons, and armor. Avoid...",1,1.071705,ad4965fd-670b-4d7b-ade3-e126ac213d9c,18aa634e-298a-41a2-92e5-5f6b0371e3b4


### Running over Different pieces of Data

In [ ]:
# passing initial dataset to as_of didn't work, so i found the list of dataset versions i had and picked the timeestamp of the first version manually

In [11]:
versions = list(client.list_dataset_versions(dataset_name=dataset_name))
print(versions)


[DatasetVersion(tags=['latest'], as_of=datetime.datetime(2025, 10, 4, 18, 47, 18, 53445, tzinfo=datetime.timezone.utc)), DatasetVersion(tags=[], as_of=datetime.datetime(2025, 10, 4, 18, 47, 18, 51957, tzinfo=datetime.timezone.utc)), DatasetVersion(tags=[], as_of=datetime.datetime(2025, 10, 4, 18, 47, 18, 50609, tzinfo=datetime.timezone.utc)), DatasetVersion(tags=[], as_of=datetime.datetime(2025, 10, 4, 18, 40, 19, 786727, tzinfo=datetime.timezone.utc)), DatasetVersion(tags=[], as_of=datetime.datetime(2025, 10, 4, 18, 31, 0, 174986, tzinfo=datetime.timezone.utc))]


In [12]:
import datetime

examples = list(client.list_examples(
    dataset_name=dataset_name,
    as_of=datetime.datetime(2025, 10, 4, 18, 31, 0, 174986, tzinfo=datetime.timezone.utc)  # initial dataset timestamp
))

evaluate(
    target_function,
    data=examples,
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)


View the evaluation results for experiment: 'initial dataset version-9b6e643a' at:
https://smith.langchain.com/o/55075c61-fb01-4762-bfb5-95638bc62c05/datasets/d5760ca9-0eaa-493c-909f-382fe0e64af3/compare?selectedSessions=ade55c2b-7eda-42fe-9b2d-cf8e972cfbad




10it [00:13,  1.39s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I fight mobs?,I don't have enough information to answer that...,None,"Use swords or bows. Keep your distance, attack...",1,1.449410,2eb6643d-1011-4ab9-b92a-24f5c7a61441,4eb04838-b109-4df1-9670-77945bd00a4a
1,What’s the difference between Survival and Cre...,I don't know the answer to that question.,None,"Survival mode requires gathering resources, ma...",1,1.280302,34aaed32-1485-4133-a1a4-73021114d881,32830a3c-84ae-4fed-8137-a0b23042a3bf
2,How do I mine for diamonds?,I don't have information on how to mine for di...,None,"Dig down to levels 11–12 underground, mine car...",1,1.137116,3f3ca99b-3f17-4bd8-bdfd-bce4cbc03086,4916e542-72ad-4648-afd1-8386fe04b620
3,How do I start playing Minecraft?,I don't have information related to playing Mi...,None,"First, create a new world in Survival or Creat...",1,1.167751,98bfec98-72ed-4362-b84c-45d6910fe51e,a5a2a9c6-6398-4993-a1ce-a95995b94196
4,How do I craft items?,I don't have information on how to craft items...,None,"Open your inventory (E), place materials in th...",1,1.169803,a81fb131-6a89-44fe-9596-5ae2e39f525f,a18050b1-a37f-4182-86ee-06c10fed3540
5,How do I explore safely?,I do not have enough information to answer the...,None,"Carry food, torches, weapons, and armor. Avoid...",1,1.444942,ad4965fd-670b-4d7b-ade3-e126ac213d9c,9116f8c0-c64b-483a-9a61-d2797e4949cc
6,How do I make a shelter?,"I'm sorry, I don't have information on how to ...",None,"Use wood, dirt, or stone to build walls and a ...",1,1.351711,ad72162c-fbf5-4992-b3e5-5e55a69120fe,8966be46-6852-461a-9a54-f3edf1c18474
7,How do I grow food?,I don't have the specific information on how t...,None,"Plant seeds from wheat, carrots, or potatoes. ...",1,1.207397,b6c0c647-0b51-4db9-bfee-f46f45d2f28f,f9568a95-2170-451e-bfb9-b762143c683a
8,How do I gather resources?,"To gather resources with LangSmith, you can as...",None,"Punch trees for wood, mine stone and ores with...",0,1.425372,e0a03c62-0e48-4e88-9c40-ec4cd8d2da33,fca29b90-443d-4663-b443-07287935685e
9,How do I survive the first night?,I don't have enough information to answer that...,None,"Gather wood and stone quickly, craft basic too...",1,1.661700,fc573bea-8ed6-4369-ae1d-e2e527c94bd7,a76fc70b-d7d0-45ec-8655-c41d485e8bda


##### Dataset Split



In [ ]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial Examples"]),  
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial Examples split"
)

View the evaluation results for experiment: 'Crucial Examples split-38594675' at:
https://smith.langchain.com/o/55075c61-fb01-4762-bfb5-95638bc62c05/datasets/d5760ca9-0eaa-493c-909f-382fe0e64af3/compare?selectedSessions=39162c1d-bfc0-4e12-af9e-9e113f6eb462




8it [00:16,  2.05s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I mine for diamonds?,I don't have information on how to mine for di...,None,"Dig down to levels 11–12 underground, mine car...",1,1.263867,3f3ca99b-3f17-4bd8-bdfd-bce4cbc03086,310b2f4b-c8e4-4377-8985-ae6eebe38051
1,How do I start playing Minecraft?,I don't know the answer to that question.,None,"First, create a new world in Survival or Creat...",1,2.896858,98bfec98-72ed-4362-b84c-45d6910fe51e,b8b13344-36bb-4df8-88bd-7bc1770d21cb
2,How do I craft items?,I don't have information on how to craft items...,None,"Open your inventory (E), place materials in th...",1,3.862685,a81fb131-6a89-44fe-9596-5ae2e39f525f,d66efb00-19fc-4e8f-bbfe-f26a93f95226
3,How do I explore safely?,"I'm sorry, I don't have the information to ans...",None,"Carry food, torches, weapons, and armor. Avoid...",1,1.440094,ad4965fd-670b-4d7b-ade3-e126ac213d9c,1d604461-bc66-43b7-8506-b1cfb2305c13
4,How do I make a shelter?,I don't have information on making a shelter i...,None,"Use wood, dirt, or stone to build walls and a ...",1,1.898921,ad72162c-fbf5-4992-b3e5-5e55a69120fe,8a3189ce-93e3-490d-98ef-85b9c15257a1
5,How do I grow food?,I don't have information on growing food withi...,None,"Plant seeds from wheat, carrots, or potatoes. ...",1,1.719755,b6c0c647-0b51-4db9-bfee-f46f45d2f28f,36130136-b0a6-40be-94c4-cab160aa3bb8
6,How do I gather resources?,"To gather resources, you can assign tags to th...",None,"Punch trees for wood, mine stone and ores with...",0,1.688910,e0a03c62-0e48-4e88-9c40-ec4cd8d2da33,76db3727-1365-4c55-95eb-d40d1ab90e44
7,How do I survive the first night?,I don't have enough information to answer the ...,None,"Gather wood and stone quickly, craft basic too...",1,1.053676,fc573bea-8ed6-4369-ae1d-e2e527c94bd7,34f22a0d-a160-45ab-9585-95f8db225247


##### Specific Data Points



In [14]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   
            "3d32f633-70f6-478b-b768-a1e163e021e7",
            "b5d20f18-7ac5-4d5f-a9a4-2e079cf688a5"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-9a90b4ba' at:
https://smith.langchain.com/o/55075c61-fb01-4762-bfb5-95638bc62c05/datasets/d5760ca9-0eaa-493c-909f-382fe0e64af3/compare?selectedSessions=114aea20-9cf2-4a46-9b75-df71171d987f




2it [00:03,  1.72s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I make a map?,I don't have specific information on creating ...,None,"To create a map in Minecraft, first craft a co...",1,1.508027,3d32f633-70f6-478b-b768-a1e163e021e7,82d34c07-5e1e-4160-9f4c-b36e5f97f0f2
1,How do I enchant items in Minecraft?,"I'm sorry, but the provided context does not c...",None,"To enchant items in Minecraft, you need to cra...",1,1.359048,b5d20f18-7ac5-4d5f-a9a4-2e079cf688a5,7f762688-d962-41d5-96a8-f3bda914c9b4


### Other Parameters

In [15]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-a19f8352' at:
https://smith.langchain.com/o/55075c61-fb01-4762-bfb5-95638bc62c05/datasets/d5760ca9-0eaa-493c-909f-382fe0e64af3/compare?selectedSessions=81347451-e163-4960-97b1-251abd323ad6




28it [02:21,  5.04s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I mine for diamonds?,I don't know the answer to that question.,None,"Dig down to levels 11–12 underground, mine car...",1,1.407376,3f3ca99b-3f17-4bd8-bdfd-bce4cbc03086,0946c3f0-b147-42eb-b92f-1ed9b9c5bee0
1,How do I start playing Minecraft?,I don't know the answer to that question.,None,"First, create a new world in Survival or Creat...",1,1.059579,98bfec98-72ed-4362-b84c-45d6910fe51e,3bb4d76f-109f-4c67-a876-c29017a43d7f
2,How do I craft items?,I don't have information on crafting items bas...,None,"Open your inventory (E), place materials in th...",1,1.358229,a81fb131-6a89-44fe-9596-5ae2e39f525f,faaffb11-5d22-4783-a265-7065bf327f7c
3,How do I explore safely?,I don't have enough information to answer the ...,None,"Carry food, torches, weapons, and armor. Avoid...",1,1.209524,ad4965fd-670b-4d7b-ade3-e126ac213d9c,2cc27dc4-94fd-45e3-89e1-b75e58fabb15
4,How do I make a shelter?,I don't have information on making a shelter b...,None,"Use wood, dirt, or stone to build walls and a ...",1,1.674421,ad72162c-fbf5-4992-b3e5-5e55a69120fe,44752589-a486-4d09-972f-7e20059eb068
5,How do I grow food?,"I'm sorry, I couldn't find information on how ...",None,"Plant seeds from wheat, carrots, or potatoes. ...",0,46.935834,b6c0c647-0b51-4db9-bfee-f46f45d2f28f,1673aecb-4da6-45a7-8fde-f6b5dc35037c
6,How do I gather resources?,"To gather resources in LangSmith, you can use ...",None,"Punch trees for wood, mine stone and ores with...",0,2.146239,e0a03c62-0e48-4e88-9c40-ec4cd8d2da33,75f5cefa-dba6-458d-982a-48c97a97fe52
7,How do I survive the first night?,I don't know the answer to that question as it...,None,"Gather wood and stone quickly, craft basic too...",1,1.279205,fc573bea-8ed6-4369-ae1d-e2e527c94bd7,259dbde3-9323-4a97-80f9-8b2600602f0e
8,How do I make a map?,I don't have information on making a map based...,None,"To create a map in Minecraft, first craft a co...",1,1.430726,3d32f633-70f6-478b-b768-a1e163e021e7,2e6ca0ff-9fa9-4820-8c99-f985e8ae8796
9,What are Nether portals used for?,I don't have information on Nether portals or ...,None,Nether portals are used to access the Nether d...,1,1.356794,0220929e-9638-485b-bbaa-f5a5bfaf2b63,28aa87ff-6700-4cc7-8f30-70b8ff8c05b9


##### Concurrency


In [16]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=2,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-8abdf1ca' at:
https://smith.langchain.com/o/55075c61-fb01-4762-bfb5-95638bc62c05/datasets/d5760ca9-0eaa-493c-909f-382fe0e64af3/compare?selectedSessions=b5c6e698-2161-451c-8f14-7821e2f72c9c




14it [00:09,  1.46it/s]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I start playing Minecraft?,I don't have information about how to start pl...,None,"First, create a new world in Survival or Creat...",1,1.222367,98bfec98-72ed-4362-b84c-45d6910fe51e,a16c2662-6b5c-452a-ae37-fb2f418e737f
1,How do I mine for diamonds?,I don't know the answer.,None,"Dig down to levels 11–12 underground, mine car...",1,1.416157,3f3ca99b-3f17-4bd8-bdfd-bce4cbc03086,6636523b-5654-4bbd-92e9-6d1c7a841771
2,How do I craft items?,"I'm sorry, I couldn't find specific informatio...",None,"Open your inventory (E), place materials in th...",1,1.068581,a81fb131-6a89-44fe-9596-5ae2e39f525f,9632dbd0-3ad9-4823-90f2-e96002efadfe
3,How do I explore safely?,I don't know.,None,"Carry food, torches, weapons, and armor. Avoid...",1,1.150191,ad4965fd-670b-4d7b-ade3-e126ac213d9c,b0b8c8ca-e75a-4753-9705-76da6af35775
4,How do I make a shelter?,I don't have information on making a shelter i...,None,"Use wood, dirt, or stone to build walls and a ...",1,1.097551,ad72162c-fbf5-4992-b3e5-5e55a69120fe,5e77c9ab-54d4-4c61-aab5-ae4365146b24
5,How do I grow food?,I don't have information on growing food.,None,"Plant seeds from wheat, carrots, or potatoes. ...",1,1.614920,b6c0c647-0b51-4db9-bfee-f46f45d2f28f,2252bf5f-5c20-46d9-866d-1024e2be3de1
6,How do I gather resources?,"To gather resources in LangSmith, you can assi...",None,"Punch trees for wood, mine stone and ores with...",0,1.636968,e0a03c62-0e48-4e88-9c40-ec4cd8d2da33,852919ae-aa4e-480c-a672-34845651172f
7,How do I survive the first night?,I don't have enough information to answer the ...,None,"Gather wood and stone quickly, craft basic too...",1,1.094165,fc573bea-8ed6-4369-ae1d-e2e527c94bd7,841c13b6-9d5d-43bb-b2a3-da80fe818376
8,What are Nether portals used for?,I don't have information on Nether portals.,None,Nether portals are used to access the Nether d...,1,0.954631,0220929e-9638-485b-bbaa-f5a5bfaf2b63,f9a12a16-3b2b-45ba-8236-201ef4a47a83
9,How do I make a map?,I don't have information on making a map based...,None,"To create a map in Minecraft, first craft a co...",1,1.218495,3d32f633-70f6-478b-b768-a1e163e021e7,aa903c27-2be4-4394-a8b7-db236dddd664


## Metadata 



In [19]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={ 
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-89a7a188' at:
https://smith.langchain.com/o/55075c61-fb01-4762-bfb5-95638bc62c05/datasets/d5760ca9-0eaa-493c-909f-382fe0e64af3/compare?selectedSessions=2b8e2732-e874-4849-8fd9-bcdb1796707b




14it [00:33,  2.38s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I mine for diamonds?,I don't know the answer to that question.,None,"Dig down to levels 11–12 underground, mine car...",1,1.694399,3f3ca99b-3f17-4bd8-bdfd-bce4cbc03086,e8872b0c-1367-467f-9912-d780c211f83e
1,How do I start playing Minecraft?,I don't have information on how to start playi...,None,"First, create a new world in Survival or Creat...",1,1.124630,98bfec98-72ed-4362-b84c-45d6910fe51e,cdcffa34-7073-4fc9-9810-7bc9fb8a4c63
2,How do I craft items?,I don't have the specific information on craft...,None,"Open your inventory (E), place materials in th...",1,1.177425,a81fb131-6a89-44fe-9596-5ae2e39f525f,4c3df4a0-d617-4c5c-9636-3b35ecb5e478
3,How do I explore safely?,I don't know the answer to that question based...,None,"Carry food, torches, weapons, and armor. Avoid...",1,1.159974,ad4965fd-670b-4d7b-ade3-e126ac213d9c,2fafe040-9c86-49a8-a824-11857d13def4
4,How do I make a shelter?,I don't know the answer to that question based...,None,"Use wood, dirt, or stone to build walls and a ...",1,1.130436,ad72162c-fbf5-4992-b3e5-5e55a69120fe,614e5828-a28f-43aa-bdb7-e07fb114f457
5,How do I grow food?,"I'm sorry, I don't have information on how to ...",None,"Plant seeds from wheat, carrots, or potatoes. ...",1,1.176264,b6c0c647-0b51-4db9-bfee-f46f45d2f28f,efaf96de-935e-4668-be7a-503c1adaedb7
6,How do I gather resources?,"To gather resources, you can assign tags to re...",None,"Punch trees for wood, mine stone and ores with...",0,1.572289,e0a03c62-0e48-4e88-9c40-ec4cd8d2da33,5d850fd6-2a68-446a-a0d7-eec553c6953b
7,How do I survive the first night?,I don't know the answer to that question.,None,"Gather wood and stone quickly, craft basic too...",1,15.724760,fc573bea-8ed6-4369-ae1d-e2e527c94bd7,614e0f50-abd2-451e-9da4-36f343b94aa2
8,How do I make a map?,I don't have information on making a map based...,None,"To create a map in Minecraft, first craft a co...",1,1.102535,3d32f633-70f6-478b-b768-a1e163e021e7,afcb1c98-d2e6-42b6-a60a-caf885f6b042
9,What are Nether portals used for?,I don't know the answer to that question.,None,Nether portals are used to access the Nether d...,1,1.159717,0220929e-9638-485b-bbaa-f5a5bfaf2b63,c49c0191-75fc-47f4-92f4-1bbe15cc04cd
